## torchtext - korean

In [1]:
!pip install python-mecab-ko

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import urllib.request
import pandas as pd

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7fcfa1890b20>)

In [4]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [5]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
train_df.dropna(inplace=True)

In [7]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [8]:
test_df.dropna(inplace=True)

In [9]:
print('훈련 데이터 샘플의 개수 : {}'.format(len(train_df)))
print('테스트 데이터 샘플의 개수 : {}'.format(len(test_df)))

훈련 데이터 샘플의 개수 : 149995
테스트 데이터 샘플의 개수 : 49997


In [10]:
from torchtext import data # torchtext.data 임포트
from mecab import MeCab

/home/ubuntu/.local/lib/python3.10/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [11]:
# Mecab을 토크나이저로 사용

def tokenizer(text):
    mecab = MeCab()
    tokens = mecab.morphs(text)
    return tokens

In [12]:
from typing import List, Tuple

def yield_tokens(data_iter: pd.Series) -> List[str]:
    for text in data_iter:
        yield tokenizer(text)

In [13]:
from torchtext.vocab import build_vocab_from_iterator

text_vocab = build_vocab_from_iterator(yield_tokens(train_df['document']), specials=['<unk>'])
text_vocab.set_default_index(text_vocab['<unk>'])

/home/ubuntu/.local/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/ubuntu/.local/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [14]:
import os
import random
import numpy as np
import torch
import torchvision

In [15]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [16]:
%matplotlib inline

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

In [17]:
import pandas as pd
import torch
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple

class NSMCDataset(Dataset):
    def __init__(self, df: pd.DataFrame, text_vocab):
        self.df = df
        self.text_vocab = text_vocab

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item  =self.df.iloc[idx]
        return {
            'id': item['id'],
            'document': torch.tensor([self.text_vocab[token] for token in tokenizer(item['document'])]),
            'label': torch.tensor(int(item['label']))
        }  

In [18]:
from torch.utils.data import random_split

train_data = NSMCDataset(train_df, text_vocab)
test_data = NSMCDataset(test_df, text_vocab)

In [19]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 149995
테스트 샘플의 개수 : 49997


In [20]:
print(train_data[0])

{'id': 9976970, 'document': tensor([ 40,  85, 938,   1,   1,  47, 243,  27,  41, 766]), 'label': tensor(0)}


In [21]:
print('단어 집합의 크기 : {}'.format(len(text_vocab)))

단어 집합의 크기 : 53980


In [22]:
print(text_vocab.get_stoi())

{'￣∇￣)': 53978, '＾＾': 53975, '！！！': 53972, '律': 53970, '힛처': 53963, '힘준': 53962, '힘써서': 53959, '힘든가': 53956, '힘내기': 53955, '힘겨웠': 53954, '힘겨워': 53953, '힐신': 53951, '힐러리스웽크': 53950, '힉콕스': 53946, '히히히히히': 53944, '히프': 53943, '히포크라테스': 53942, '히치': 53939, '히지': 53938, '히죽히죽': 53937, '히잉': 53935, '히스토리': 53932, '히사코': 53931, '히브리': 53928, '히무로': 53926, '히메나': 53924, '히마': 53923, '히로시마': 53921, '히로미': 53919, '히뜩': 53917, '히나': 53913, '흳': 53912, '흰개': 53910, '희재': 53909, '희기': 53899, '희곡': 53898, '흩어져서': 53893, '흩뿌렸': 53891, '흥겨워서': 53882, '흥건': 53881, '흠좀무': 53880, '흙모래': 53877, '흘쩍': 53876, '흘러야': 53872, '흘러버렸': 53871, '흘러내리': 53869, '흘러나왔': 53868, '흘러나와': 53867, '흘깃': 53865, '흔했': 53864, '흔해져서': 53863, '흔해빠졌': 53862, '흔한데': 53861, '흔쾌히': 53860, '흔쾌': 53859, '흔이': 53858, '흑흑흑흑': 53854, '흑적색': 53851, '흑암': 53848, '흐흐흐흐': 53841, '흐트림': 53840, '흐트러': 53838, '흐엉': 53837, '흐린다는': 53835, '흐른다는': 53831, '흐륵': 53830, '흐드러지': 53828, '흐느적거리': 53827, '흉식호흡': 53821, '흉물': 53820, '흉기': 53818, '휼륭하다':

In [23]:
print(text_vocab.get_itos())

['<unk>', '.', '이', '는', '영화', '다', '고', '하', '도', '의', '가', '은', '에', '을', '보', '한', '..', '게', ',', '들', '!', '지', '를', '있', '없', '?', '좋', '나', '었', '만', '는데', '너무', '봤', '적', '안', '정말', '로', '음', '으로', '것', '아', '네요', '재밌', '어', '점', '같', '지만', '진짜', '했', '에서', '기', '네', '않', '거', '았', '수', '되', '면', 'ㅋㅋ', '과', '말', '연기', '인', '잘', '최고', '주', '내', '~', '평점', '이런', '어요', '던', '와', '할', '왜', '1', '겠', '해', '스토리', '습니다', 'ㅋㅋㅋ', '...', '드라마', '생각', '아니', '더', '그', '싶', '사람', '듯', '감동', '때', '함', '배우', '본', '까지', '좀', '뭐', '볼', '알', '만들', '내용', '감독', '보다', '라', '재미', '그냥', '지루', '시간', '재미있', '중', '년', '잼', '10', '재미없', '였', '쓰레기', '사랑', '못', '냐', '서', '2', '라고', '야', '니', '면서', '번', '다시', '나오', '작품', '이거', '하나', '줄', '해서', '개', '남', '정도', '마지막', '끝', '이건', '임', '액션', 'ㅋ', '3', '입니다', '기대', '건', '완전', '라는', '분', '다는', '참', '많', '아깝', '처음', '장면', '대', '다가', '으면', '지금', '모르', '이렇게', "'", '편', '일', '이게', '돈', '최악', '성', '느낌', '시', '이야기', '별로', '된', '봐도', '님', '어서', '애', '전', 'ㅠㅠ', '넘', '인데'

In [24]:
batch_size = 5

In [25]:
def collate_batch(batch):
    label_list, text_list = [], []
    for item in batch:
        label_list.append(item['label'])
        text_list.append(item['document'])
    
    text_list = torch.nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=text_vocab['<pad>'])
    label_list = torch.tensor(label_list, dtype=torch.int64)
    return text_list, label_list


In [26]:
train_loader = DataLoader(dataset=train_data, batch_size = batch_size, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(dataset=test_data, batch_size = batch_size, shuffle=True, collate_fn=collate_batch)

In [27]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 29999
테스트 데이터의 미니 배치 수 : 10000


In [28]:
for document, label in train_loader:
    print(document)
    print(label)
    break

tensor([[ 3542,     7,    17,  1997,    43,    66,     3,  1161,    15,   613,
          2225,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [   68,    14,     6,    32,   157,   305,    15,     4,     1,  6091,
          2418, 11686,    72,     2,     4,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  105,    24,    79,     1,   114,    79,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     

In [29]:
print(document.shape)
print(label.shape)

torch.Size([5, 46])
torch.Size([5])
